### Implementing simple Chatbot Using LangGraph

In [ ]:
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END

## Reducers
from typing import Annotated
from langgraph.graph.message import add_messages

In [ ]:
# State defines the shared data structure that flows between nodes
class State(TypedDict):
    messages:Annotated[list,add_messages]

Why Annotated[list, add_messages]?
* Annotated is a Python typing feature that lets you attach metadata to a type.
* Here, the metadata add_messages tells LangGraph:
</br>“When multiple message lists come together, combine them intelligently (not overwrite).”

So if one node outputs a list of messages, and another adds new ones, add_messages will append them instead of replacing.

This enables conversation continuity — your chatbot can accumulate turns of dialogue.

---
* State = A container that holds your data
* messages = A field inside that container (like a variable)
* list = Says "messages is a list"
* add_messages = Says "when you update messages, ADD to the list instead of replacing it"

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")


In [ ]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")
llm.invoke("Hello")

In [ ]:
from langchain_groq import ChatGroq
# Initializing the Language Models
llm_groq=ChatGroq(model="qwen-qwq-32b")
llm_groq.invoke("Hey I am Jhon and i like to play cricket")

### We Will start With Creating Nodes

In [ ]:
def superbot(state:State):
    return {"messages":[llm_groq.invoke(state['messages'])]}

* superbot() is a LangGraph node function.
* Input: the current graph state, which includes all previous messages.
* It sends the user’s messages to the Groq LLM (llm_groq.invoke()).
* The LLM returns an AIMessage object containing the chatbot’s response.
* That message is returned as a list (so it can be appended by add_messages).

In [ ]:
graph=StateGraph(State)

## node
graph.add_node("SuperBot",superbot)
## Edges

graph.add_edge(START,"SuperBot")
graph.add_edge("SuperBot",END)


graph_builder=graph.compile()


## Display
from IPython.display import Image, display
display(Image(graph_builder.get_graph().draw_mermaid_png()))

In [ ]:
## Invocation

graph_builder.invoke({'messages':"Hi,My name is Jhon And I like cricket"})

| Component                       | Role          | Description                                          |
| ------------------------------- | ------------- | ---------------------------------------------------- |
| `TypedDict`                     | State schema  | Defines keys and types of the graph’s shared data    |
| `Annotated[list, add_messages]` | Reducer       | Tells LangGraph how to merge message lists           |
| `superbot()`                    | Node function | Uses an LLM to process messages and generate replies |
| `StateGraph(State)`             | Graph         | Container for nodes and edges with the given schema  |
| `add_node()`                    | Method        | Registers a function as a graph node                 |
| `add_edge()`                    | Method        | Connects nodes (START → Node → END)                  |
| `compile()`                     | Method        | Validates and builds the graph                       |
| `invoke()`                      | Method        | Runs the graph once, returns the output state        |
